In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root='./data', train=True, transform=trans, download=True)
test_set = dset.MNIST(root='./data', train=False, transform=trans)
batch_size = 32


train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

/opt/conda/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
class AlexNet(nn.Module):   
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),   
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,num),
         
        )
    
    def forward(self, x):

        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x
    

In [7]:
def train(epoch, data_size = None):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if data_size is not None:
            if batch_idx*batch_size > data_size:
                break
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
#         loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('\rEpoch: {} {:.0f}%\t     Loss: {:.6f}'.format(
                epoch,
                100. * batch_idx / len(train_loader), loss.item()), end='')

In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    
    
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    acc=100. * float(correct.to(torch.device('cpu')).numpy())
    print('\nTest result: Average loss: {:.4f}, Accuracy: {:.4f}%\n'.format(
        test_loss, acc / len(test_loader.dataset)))
    
    test_accuracy.append(acc / len(test_loader.dataset))

In [10]:
import time

for sz in [5000,10000,50000]:
    model = AlexNet()
    #if torch.cuda.is_available():
    #    model.cuda()

    optimizer = optim.SGD(model.parameters(), lr=0.01)

    start_time = time.time()
    train_losses = []
    test_losses =[]
    test_accuracy = []
    for epoch in range(1, 15):
        train(epoch,sz)
        test()

    print(sz,time.time() - start_time)

    
    

Epoch: 1 5%	     Loss: 2.302842

/tmp/ipykernel_8490/3364036503.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test result: Average loss: 2.3017, Accuracy: 11.3500%

Epoch: 2 5%	     Loss: 2.295021
Test result: Average loss: 2.3015, Accuracy: 11.3500%

Epoch: 3 5%	     Loss: 2.312072
Test result: Average loss: 2.3013, Accuracy: 11.3500%

Epoch: 4 5%	     Loss: 2.299050
Test result: Average loss: 2.3012, Accuracy: 11.3500%

Epoch: 5 5%	     Loss: 2.318035
Test result: Average loss: 2.3011, Accuracy: 11.3500%

Epoch: 6 5%	     Loss: 2.293981
Test result: Average loss: 2.3009, Accuracy: 11.3500%

Epoch: 7 5%	     Loss: 2.307189
Test result: Average loss: 2.3009, Accuracy: 11.3500%

Epoch: 8 5%	     Loss: 2.302180
Test result: Average loss: 2.3008, Accuracy: 11.3500%

Epoch: 9 5%	     Loss: 2.310660
Test result: Average loss: 2.3007, Accuracy: 11.3500%

Epoch: 10 5%	     Loss: 2.319592
Test result: Average loss: 2.3006, Accuracy: 11.3500%

Epoch: 11 5%	     Loss: 2.298089
Test result: Average loss: 2.3008, Accuracy: 11.3500%

Epoch: 12 5%	     Loss: 2.315783
Test result: Average loss: 2.3006, Accu

In [11]:
1000 125.25717401504517
3000 189.90918970108032
5000 255.80986189842224
10000 415.30926871299744
50000 1756.6890790462494

SyntaxError: invalid syntax (4219509764.py, line 1)

In [11]:
!pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 14.9 MB 2.3 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1


In [13]:
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 753.4 MB 602 kB/s eta 0:00:011    |████▏                           | 97.3 MB 621 kB/s eta 0:17:36     |██████▍                         | 149.8 MB 608 kB/s eta 0:16:32     |███████▋                        | 179.4 MB 256 kB/s eta 0:37:23     |████████▍                       | 196.8 MB 53.8 MB/s eta 0:00:11     |████████▋                       | 201.7 MB 53.8 MB/s eta 0:00:11     |█████████▌                      | 223.6 MB 161 kB/s eta 0:54:48     |██████████                      | 235.3 MB 153 kB/s eta 0:56:11     |█████████████████▍              | 409.3 MB 25.3 MB/s eta 0:00:14     |█████████████████▌              | 411.4 MB 25.3 MB/s eta 0:00:14     |██████████████████▊             | 441.8 MB 283 kB/s eta 0:18:19     |████████████████████▉           | 489.3 MB 146 kB/s eta 0:29:58     |█████████████████████           | 493.7 MB 146 kB/s eta 0:29:27     |█████████████████████

In [2]:
!pip install torchvision==0.10.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 22.1 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 831.4 MB 18.2 MB/s eta 0:00:01    |▌                               | 12.7 MB 68.1 MB/s eta 0:00:13     |▊                               | 17.4 MB 68.1 MB/s eta 0:00:12     |███▏                            | 82.7 MB 62.9 MB/s eta 0:00:12     |█████████▊                      | 253.1 MB 37.8 MB/s eta 0:00:16     |██████████▌                     | 272.6 MB 37.8 MB/s eta 0:00:15     |████████████▋                   | 326.4 MB 28.4 MB/s eta 0:00:18     |█████████████                   | 340.1 MB 28.4 MB/s eta 0:00:18     |█████████████▉                  | 360.1 MB 177 kB/s eta 0:44:18     |██████████████▋                 | 378.5 MB 177 kB/s eta 0:42:34     |███████████████▏                | 394.9 MB 177 kB/s eta 0:41:02     |███████████████████▋            | 510.4 MB 29.1 MB/s eta 0:00:12     |███████████████████